In [19]:
import numpy as np
from itertools import product
import math
import random
import copy

According to Hinton's "A Practical Guide to Training Restricted Boltzmann Machines"

Initialize J matrix to zero mean and 0.01 sd. The diagonal will be 0.

Initizlize bias b_i  of visible unit i to log[pi/(1−pi)]

#### ISING MODEL

In [43]:
class IsingModel:
    def __init__(self, C_mode='corr', lr=0.01):
        self.lr = lr
        
        # Possible values the spins can take
        self.spin_values = [-1,1]
        # self.gaussian_scale = 0.01
        self.gaussian_scale = 1/math.sqrt(math.sqrt(4))
        self.C_mode = C_mode
    
    def fit_activations(self, activations_list):
        
        # Create a ndarray from the data
        self.data_activations_matrix = np.array(activations_list)
        # Calculate the shape of the matrix and save useful variables
        self.num_spins = self.data_activations_matrix.shape[1]
        self.num_samples_data = self.data_activations_matrix.shape[0]
        
        # Calculate the true data expectations
        # Calculate the mean
        self.data_mean = np.mean(self.data_activations_matrix, axis=0)
        
        # Calculate the correlation
        # Calculate the correlation
        if(self.C_mode == 'cov'):
            self.data_correlation = self.calculate_cov(self.data_activations_matrix)
        else:
            self.data_correlation = self.calculate_corr(self.data_activations_matrix)

        
        #Create random H and J matrices
        self.create_H_J()
        
         
    def fit_m_c(self, m, C):
        self.data_mean = m
        self.data_correlation = C
        self.num_spins = self.data_correlation.shape[1]
        
        self.create_H_J()

    
    def create_H_J(self, gaussian_scale = None):
        if gaussian_scale is None:
            gaussian_scale = self.gaussian_scale
            
        # Calculate the initial J matrix. Fill only the upper diagonal
        # Filling with Normal(0, 0.01)
        # np.random.seed(3)
        self.J = np.random.normal(loc=0.0, scale=gaussian_scale, size=(self.num_spins, self.num_spins))
        self.lind = np.tril_indices_from(self.J)
        self.J[self.lind]=0
                
        # Calculate the initial H vector, each entry proportional to its probability (according to Hinton)
        #self.H = np.sum(self.data_activations_matrix, axis=0)/ float(self.num_samples_data)
        # Try with random H
        self.H = np.random.normal(loc=0.0, scale=gaussian_scale, size=self.num_spins)
    
    def set_H_J(self):
        self.H = H
        self.J = J        
        
    # Train the exact model
    def train(self, max_epochs = 500):
        
        totalParamVariation = math.inf
        stopCondition = 0.0001
        
        epoch = 1
        
        while totalParamVariation > stopCondition and epoch < max_epochs:
            # Calculate P(sigma) for every possible combination in the model

            prob_dict = {}
            for sigma in product(self.spin_values, repeat=self.num_spins):
                prob_dict[sigma] = math.exp(-self.calculate_energy(sigma))

            # Compute the statistics
            if(self.C_mode == 'cov'):   
                model_mean, model_correlation = self.get_statistics_cov(prob_dict, False)
            else:
                model_mean, model_correlation = self.get_statistics_corr(prob_dict)

            # Calculate the step size for every J_{ij}
            stepJ = self.lr * (self.data_correlation - model_correlation)
            # Take the step
            oldJ = copy.deepcopy(self.J)
            self.J = self.J + stepJ
            self.J[self.lind]=0

            # Calculate the step size for every H_{i}
            stepH = self.lr * (self.data_mean - model_mean)
            # Calculate the variation of J for the termination condition
            # Take the step
            oldH = copy.deepcopy(self.H)
            self.H = self.H + stepH
            
            diffH = np.sum(np.absolute(self.H - oldH))
            diffJ = np.sum(np.sum(np.absolute(self.J - oldJ)))
            totalParamVariation = diffJ + diffH
            
            if epoch%100 == 0:
                print('Epoch', epoch, 'TotalParamVariation', round(totalParamVariation, 8))
                print(model_correlation, 'Corr Model')
                print(self.data_correlation, 'Corr Data')

                print(model_mean, 'Average Model')
                print(self.data_mean, 'Average Data')
                print()
            
            epoch += 1
            
        print('Final stats. TotalParamVariation', round(totalParamVariation, 8))
        print('Total epochs', epoch-1)
        print(model_correlation, 'C Model')
        print(self.data_correlation, 'C Data')

        print(model_mean, 'Average Model')
        print(self.data_mean, 'Average Data')
        print()

        print(self.H, 'H')
        print(self.J, 'J')

    # Calculate the energy of the model's parameters given a sigma state
    def calculate_energy(self, sigma):
        energy = 0
        for i in range(0, self.num_spins):
            for j in range(i+1, self.num_spins):
                energy += sigma[i]*sigma[j]*self.J[i][j]

        for i in range(0, self.num_spins):
            energy += sigma[i]*self.H[i]

        return - energy
    
    def get_delta_energy(self, i):
        #delta_energy = 2*(self.H[spin_to_flip]*self.sigma[spin_to_flip])
        
        delta_energy= 2 * (self.H[i]*self.sigma[i] \
                           + self.sigma[i] * np.dot(self.J[i, :], self.sigma) \
                           + self.sigma[i] * np.dot(self.J[:, i], self.sigma))
        
        return delta_energy

    # Calculates the correlation of a list of activations
    def calculate_corr(self, activations_matrix):
        num_samples_data = activations_matrix.shape[0]
        
        corr_matrix = np.zeros((self.num_spins, self.num_spins))
        for sigma in activations_matrix:
            # Calculate \sigma_i * \sigma_j
            for i in range(0, self.num_spins):
                for j in range(i+1, self.num_spins):
                    corr_matrix[i][j] += sigma[i]*sigma[j]
        corr_matrix = corr_matrix / float(num_samples_data)
        
        return corr_matrix
    
    # Calculates the covariance of a list of activations
    def calculate_cov(self, activations_matrix):
        num_samples_data = activations_matrix.shape[0]
        
        cov_matrix = np.zeros((self.num_spins, self.num_spins))
        for sigma in activations_matrix:
            # Calculate \sigma_i * \sigma_j
            for i in range(0, self.num_spins):
                for j in range(i, self.num_spins):
                    cov_matrix[i][j] += (sigma[i] - self.data_mean[i])*(sigma[j] - self.data_mean[j])
        cov_matrix = cov_matrix / float(num_samples_data-1)
        
        return cov_matrix
    
    
    def get_statistics_corr(self, states_dict):
        
        # Calculate the partition function
        Z = sum(states_dict.values())
        normalized_dict = {sigma: u_prob / Z for sigma, u_prob in states_dict.items()}
        
        # Returns the magnetization and correlation from a dictionary of activations {sigma, prob}
        magnetization =  np.zeros(self.num_spins)
        corr_matrix = np.zeros((self.num_spins,self.num_spins))
        # Calculate the expectation
        for sigma, prob in normalized_dict.items():
            magnetization += np.dot(sigma, prob)
            for i in range(0, self.num_spins):
                for j in range(i+1, self.num_spins):
                    corr_matrix[i][j] += sigma[i]*sigma[j]*prob
                    
        return magnetization, corr_matrix
    
    def get_statistics_cov(self, states_dict, is_sample):        
        # Calculate the partition function
        Z = sum(states_dict.values())
        
        if is_sample:
            Z = Z-1
        
        # Returns the magnetization and correlation from a dictionary of activations {sigma, prob}
        magnetization =  np.zeros(self.num_spins)
        cov_matrix = np.zeros((self.num_spins,self.num_spins))
        # Calculate the expectation
        for sigma, u_prob in states_dict.items():
            magnetization += np.dot(sigma,u_prob/Z)
            
        for sigma, u_prob in states_dict.items():
            for i in range(0, self.num_spins):
                for j in range(i, self.num_spins):
                    cov_matrix[i][j] += (sigma[i] - magnetization[i])*(sigma[j]- magnetization[j])*u_prob
        
        cov_matrix = cov_matrix / Z
        
        return magnetization, cov_matrix
    
    def metropolis_step(self):
        # Randomly select a spin (now it can only select 0)
        spin_to_flip = np.random.randint(self.num_spins)
        delta_energy = self.get_delta_energy(spin_to_flip)

        p = math.exp(-delta_energy)
        r = random.random()

        #print(delta_energy, r, p)
        if delta_energy <= 0 or r<=p:
            self.sigma[spin_to_flip] = -self.sigma[spin_to_flip]

    def metropolis_simulation(self, max_steps, initial_burn_in_len=0, inter_burn_in=0):
        # Generate a random initial spin configuration
        self.sigma = random.choices(self.spin_values, k=self.num_spins)        
        
        # Create a dictionary to count the frequencies
        samples_dict = {}
        for i in range(0, max_steps):
            # Take a metropolis step
            self.metropolis_step()
            
            # Save the state if the burn-in phase has passed
            if i >= initial_burn_in_len and (i%inter_burn_in)==0:
                if not tuple(copy.deepcopy(self.sigma)) in samples_dict: # If the state is new, create an entry
                    samples_dict[tuple(copy.deepcopy(self.sigma))] = 1
                else: # If the state already appeared, add one
                    samples_dict[tuple(copy.deepcopy(self.sigma))] += 1
                                                 
        return samples_dict
    
    def train_metropolis(self, max_epochs = 300, simulation_len = 100, burn_in_len=0, inter_burn_in=0):
        random.seed(3)
        totalParamVariation = math.inf
        stopCondition = 0.0003
        
        epoch = 1
        
        while totalParamVariation > stopCondition and epoch < max_epochs:

            samples_dict = self.metropolis_simulation(simulation_len, burn_in_len, inter_burn_in)
        
            # Calculate the data expectations        
            if(self.C_mode == 'cov'):   
                model_mean, model_correlation = self.get_statistics_cov(samples_dict, True)
            else:
                model_mean, model_correlation = self.get_statistics_corr(samples_dict)
            
            # Calculate the step size for every J_{ij}
            stepJ = self.lr * (self.data_correlation - model_correlation)
            oldJ = copy.deepcopy(self.J)
            # Take the step
            self.J = self.J + stepJ
            self.J[self.lind]=0
            
            # Calculate the step size for every H_{i}
            stepH = self.lr * (self.data_mean - model_mean)
            # Take the step
            oldH = copy.deepcopy(self.H)
            self.H = self.H + stepH
            
            # Calculate the variation of J and H for the termination condition
            diffH = np.sum(np.absolute(self.H - oldH))
            diffJ = np.sum(np.sum(np.absolute(self.J - oldJ)))
            totalParamVariation = diffJ + diffH
            
            if epoch%50 == 0 or epoch == max_epochs-1:
                print('Epoch', epoch, 'TotalParamVariation', round(totalParamVariation, 8))
                print(model_correlation, 'Corr Model')
                print(self.data_correlation, 'Corr Data')

                print(model_mean, 'Average Model')
                print(self.data_mean, 'Average Data')
                print()
                
            epoch +=1
            
        print('Final stats. TotalParamVariation', round(totalParamVariation, 8))
        print('Total epochs', epoch-1)
        print(model_correlation, 'C Model')
        print(self.data_correlation, 'C Data')

        print(model_mean, 'Average Model')
        print(self.data_mean, 'Average Data')
        print()

        
        print(self.H, 'H')
        print(self.J, 'J')
    
    def create_initial_conditions(self, _num_spins, _num_runs, _simulation_len, _burn_in_len, _inter_burn_in):
        self.num_spins = _num_spins
        
        #Create random H and J matrices
        self.create_H_J()
        
        num_runs = _num_runs
        model_mean_array = np.empty((0,self. num_spins))
        model_correlation_array = np.empty((self.num_spins, self.num_spins, 0))
        for i in range(0,num_runs):
            samples_dict = self.metropolis_simulation(_simulation_len, _burn_in_len, _inter_burn_in)
            
            # Calculate the data expectations        
            if(self.C_mode == 'cov'):   
                model_mean, model_correlation = self.get_statistics_cov(samples_dict, True)
            else:
                model_mean, model_correlation = self.get_statistics_corr(samples_dict)
        
            model_mean_array = np.append(model_mean_array, model_mean.reshape(1, self.num_spins), axis=0)
            model_correlation_array = np.append(model_correlation_array, np.atleast_3d(model_correlation), axis=2)
            
        model_mean = np.mean(model_mean_array, axis=0)
        model_mean_std = np.std(model_mean_array, axis=0)
        model_correlation = np.mean(model_correlation_array, axis=2)
        model_correlation_std = np.std(model_correlation_array, axis=2)

        
        print(self.H, 'iniH')
        print(self.J, 'iniJ')
        
        print(model_mean, 'Mean Data')
        print(model_mean_std, 'Std Data')
        print(model_correlation, 'Corr Data')
        print(model_correlation_std, 'Corr STD')
         
        
        return self.H, self.J, model_mean, model_correlation
        
    

In [37]:
activations_list = [[1,-1,1,-1], [1,-1,1,-1], [-1,1,-1,-1], [-1,1,1,1], [1,1,1,-1], [-1,-1,1,1], [1,-1,-1,1], [1,-1,1,1]]
s_ising = IsingModel(C_mode='corr', lr=0.01)
s_ising.fit_activations(activations_list)

In [34]:
s_ising.train(10000)

Epoch 100 TotalParamVariation 0.0294756
[[ 0.         -0.00971264  0.15142682  0.37087185]
 [ 0.          0.         -0.19066101 -0.50417709]
 [ 0.          0.          0.          0.34774897]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.31749013 -0.18215312 -0.02743918  0.4137868 ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 200 TotalParamVariation 0.0181117
[[ 0.         -0.14775567  0.20740623  0.10620676]
 [ 0.          0.         -0.19157374 -0.45110525]
 [ 0.          0.          0.          0.2187944 ]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.25501236 -0.19041406  0.20518549  0.22238611] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 300 TotalParamVariation 0.00648316
[

Epoch 2200 TotalParamVariation 0.00036193
[[ 0.00000000e+00 -4.88077423e-01  2.50018946e-01 -2.38011008e-01]
 [ 0.00000000e+00  0.00000000e+00 -2.50088335e-01 -2.38010410e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.66429411e-05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 2.50089768e-01 -2.50017528e-01  4.99977199e-01  2.78757402e-05] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 2300 TotalParamVariation 0.00034538
[[ 0.00000000e+00 -4.88614893e-01  2.50019545e-01 -2.38554466e-01]
 [ 0.00000000e+00  0.00000000e+00 -2.50077943e-01 -2.38554087e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -2.24713871e-05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Dat

Epoch 4100 TotalParamVariation 0.00018931
[[ 0.00000000e+00 -4.93723270e-01  2.50013039e-01 -2.43705474e-01]
 [ 0.00000000e+00  0.00000000e+00 -2.50015714e-01 -2.43705474e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.04122468e-06]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 2.50015714e-01 -2.50013039e-01  4.99994567e-01  1.04149512e-06] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 4200 TotalParamVariation 0.00018467
[[ 0.00000000e+00 -4.93876208e-01  2.50012543e-01 -2.43859285e-01]
 [ 0.00000000e+00  0.00000000e+00 -2.50014802e-01 -2.43859285e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -8.79508426e-07]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Dat

Epoch 6600 TotalParamVariation 0.00011624
[[ 0.00000000e+00 -4.96137578e-01  2.50005351e-01 -2.46130944e-01]
 [ 0.00000000e+00  0.00000000e+00 -2.50005392e-01 -2.46130944e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.59508363e-08]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 2.50005392e-01 -2.50005351e-01  4.99998051e-01  1.59508421e-08] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 6700 TotalParamVariation 0.00011447
[[ 0.00000000e+00 -4.96196156e-01  2.50005191e-01 -2.46189724e-01]
 [ 0.00000000e+00  0.00000000e+00 -2.50005225e-01 -2.46189724e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00 -1.35129593e-08]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Dat

In [45]:
initial_ising = IsingModel(C_mode='cov')
sim_len = 20000
burn_in_len = 100
inter_burn_in = 10
num_runs = 100
num_spins = 4
H, J, model_mean, model_correlation = initial_ising.create_initial_conditions(num_spins, num_runs, sim_len, burn_in_len, inter_burn_in)

[-0.2323664   0.13245502 -0.79934924 -0.53960636] iniH
[[ 0.          0.13138246  0.46408825  1.24902115]
 [ 0.          0.          0.46837556 -0.55049773]
 [ 0.          0.          0.          0.19856825]
 [ 0.          0.          0.          0.        ]] iniJ
[-0.75895425  0.07873303 -0.77176471 -0.76451483] Mean Data
[0.02383323 0.02533757 0.01817619 0.0242221 ] Std Data
[[ 0.42421308 -0.05204849  0.1273706   0.31418487]
 [ 0.          0.99366532  0.11380457 -0.11842721]
 [ 0.          0.          0.40485125  0.09201386]
 [ 0.          0.          0.          0.41572728]] Corr Data
[[0.03616389 0.01454651 0.01895373 0.0331117 ]
 [0.         0.00406767 0.01352272 0.01623575]
 [0.         0.         0.02796773 0.01707704]
 [0.         0.         0.         0.03691138]] Corr STD


In [46]:
s_ising_corr = IsingModel(C_mode='cov', lr=0.01)
s_ising_corr.fit_m_c(model_mean, model_correlation)
s_ising_corr.train(15000)

Epoch 100 TotalParamVariation 0.02359428
[[ 0.86416774 -0.00415114 -0.21029968 -0.06786317]
 [ 0.          0.96497624  0.10950733 -0.21039262]
 [ 0.          0.          0.95291414  0.01612318]
 [ 0.          0.          0.          0.84124472]] Corr Model
[[ 0.42421308 -0.05204849  0.1273706   0.31418487]
 [ 0.          0.99366532  0.11380457 -0.11842721]
 [ 0.          0.          0.40485125  0.09201386]
 [ 0.          0.          0.          0.41572728]] Corr Data
[-0.36855429  0.18714636 -0.21699276 -0.39844106] Average Model
[-0.75895425  0.07873303 -0.77176471 -0.76451483] Average Data

Epoch 200 TotalParamVariation 0.00911178
[[ 0.547005   -0.03071185 -0.0188187   0.06989565]
 [ 0.          0.98405754  0.08456824 -0.11831984]
 [ 0.          0.          0.64533273  0.02448357]
 [ 0.          0.          0.          0.54883678]] Corr Model
[[ 0.42421308 -0.05204849  0.1273706   0.31418487]
 [ 0.          0.99366532  0.11380457 -0.11842721]
 [ 0.          0.          0.40485125  0.

Epoch 1500 TotalParamVariation 0.00054794
[[ 0.41207375 -0.05926074  0.12160832  0.31033884]
 [ 0.          0.99442671  0.11193691 -0.11221332]
 [ 0.          0.          0.41367366  0.09827172]
 [ 0.          0.          0.          0.42420119]] Corr Model
[[ 0.42421308 -0.05204849  0.1273706   0.31418487]
 [ 0.          0.99366532  0.11380457 -0.11842721]
 [ 0.          0.          0.40485125  0.09201386]
 [ 0.          0.          0.          0.41572728]] Corr Data
[-0.76676349  0.07465445 -0.76571949 -0.75881408] Average Model
[-0.75895425  0.07873303 -0.77176471 -0.76451483] Average Data

Epoch 1600 TotalParamVariation 0.00049593
[[ 0.41257518 -0.05834613  0.12180878  0.31027469]
 [ 0.          0.99427112  0.1120774  -0.11254272]
 [ 0.          0.          0.41186487  0.09736493]
 [ 0.          0.          0.          0.42385076]] Corr Model
[[ 0.42421308 -0.05204849  0.1273706   0.31418487]
 [ 0.          0.99366532  0.11380457 -0.11842721]
 [ 0.          0.          0.40485125  

Epoch 3000 TotalParamVariation 0.00015497
[[ 0.4194783  -0.05406488  0.12614632  0.3121638 ]
 [ 0.          0.99377338  0.11356051 -0.1160176 ]
 [ 0.          0.          0.40520459  0.09315803]
 [ 0.          0.          0.          0.41973069]] Corr Model
[[ 0.42421308 -0.05204849  0.1273706   0.31418487]
 [ 0.          0.99366532  0.11380457 -0.11842721]
 [ 0.          0.          0.40485125  0.09201386]
 [ 0.          0.          0.          0.41572728]] Corr Data
[-0.76191975  0.0789089  -0.7712298  -0.7617541 ] Average Model
[-0.75895425  0.07873303 -0.77176471 -0.76451483] Average Data

Epoch 3100 TotalParamVariation 0.00014421
[[ 0.41979368 -0.05396358  0.12629997  0.31226027]
 [ 0.          0.99377045  0.11360098 -0.11614689]
 [ 0.          0.          0.40511587  0.09302639]
 [ 0.          0.          0.          0.41946069]] Corr Model
[[ 0.42421308 -0.05204849  0.1273706   0.31418487]
 [ 0.          0.99366532  0.11380457 -0.11842721]
 [ 0.          0.          0.40485125  

In [4]:
s_ising.train_metropolis( max_epochs = 10000, simulation_len = 500, burn_in_len=25,  inter_burn_in=5)

Epoch 50 TotalParamVariation 0.01110526
[[ 0.         -0.30526316  0.2        -0.03157895]
 [ 0.          0.         -0.17894737 -0.03157895]
 [ 0.          0.          0.          0.09473684]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.24210526 -0.17894737  0.32631579  0.01052632] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 100 TotalParamVariation 0.01152632
[[ 0.         -0.41052632  0.2        -0.05263158]
 [ 0.          0.         -0.15789474 -0.11578947]
 [ 0.          0.          0.          0.07368421]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.11578947 -0.07368421  0.32631579  0.03157895] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 150 TotalParamVariation 0.01826316


Epoch 950 TotalParamVariation 0.00536842
[[ 0.         -0.55789474  0.17894737 -0.22105263]
 [ 0.          0.         -0.32631579 -0.17894737]
 [ 0.          0.          0.          0.01052632]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.30526316 -0.32631579  0.53684211  0.05263158] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 1000 TotalParamVariation 0.00721053
[[ 0.         -0.41052632  0.17894737 -0.28421053]
 [ 0.          0.         -0.01052632 -0.26315789]
 [ 0.          0.          0.         -0.01052632]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.13684211 -0.22105263  0.49473684  0.11578947] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 1050 TotalParamVariation 0.009157

Epoch 1850 TotalParamVariation 0.00684211
[[ 0.         -0.51578947  0.30526316 -0.28421053]
 [ 0.          0.         -0.07368421 -0.2       ]
 [ 0.          0.          0.         -0.13684211]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.36842105 -0.17894737  0.51578947  0.01052632] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 1900 TotalParamVariation 0.00831579
[[ 0.         -0.57894737  0.17894737 -0.13684211]
 [ 0.          0.         -0.22105263 -0.28421053]
 [ 0.          0.          0.         -0.11578947]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.22105263 -0.17894737  0.36842105 -0.15789474] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 1950 TotalParamVariation 0.01010

Epoch 2750 TotalParamVariation 0.00631579
[[ 0.         -0.64210526  0.32631579 -0.09473684]
 [ 0.          0.         -0.30526316 -0.17894737]
 [ 0.          0.          0.          0.03157895]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.28421053 -0.22105263  0.49473684  0.03157895] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 2800 TotalParamVariation 0.00984211
[[ 0.         -0.57894737  0.28421053 -0.24210526]
 [ 0.          0.         -0.28421053 -0.17894737]
 [ 0.          0.          0.         -0.2       ]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.13684211 -0.05263158  0.43157895 -0.17894737] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 2850 TotalParamVariation 0.00747

Epoch 3650 TotalParamVariation 0.00747368
[[ 0.         -0.30526316  0.2        -0.26315789]
 [ 0.          0.         -0.17894737 -0.43157895]
 [ 0.          0.          0.         -0.05263158]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.24210526 -0.26315789  0.45263158  0.11578947] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 3700 TotalParamVariation 0.00705263
[[ 0.         -0.49473684  0.30526316 -0.13684211]
 [ 0.          0.         -0.17894737 -0.36842105]
 [ 0.          0.          0.         -0.11578947]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.22105263 -0.30526316  0.36842105  0.01052632] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 3750 TotalParamVariation 0.00815

...

Epoch 6350 TotalParamVariation 0.007
[[ 0.         -0.53684211  0.15789474 -0.15789474]
 [ 0.          0.         -0.24210526 -0.30526316]
 [ 0.          0.          0.         -0.03157895]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.11578947 -0.32631579  0.57894737  0.09473684] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 6400 TotalParamVariation 0.00968421
[[ 0.         -0.43157895 -0.03157895 -0.15789474]
 [ 0.          0.         -0.15789474 -0.36842105]
 [ 0.          0.          0.         -0.01052632]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.17894737 -0.2         0.32631579 -0.01052632] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 6450 TotalParamVariation 0.00905263
[

Epoch 7250 TotalParamVariation 0.00615789
[[ 0.         -0.45263158  0.28421053 -0.41052632]
 [ 0.          0.         -0.2        -0.13684211]
 [ 0.          0.          0.         -0.03157895]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.17894737 -0.17894737  0.47368421 -0.01052632] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 7300 TotalParamVariation 0.00878947
[[ 0.         -0.36842105  0.36842105 -0.34736842]
 [ 0.          0.         -0.15789474 -0.28421053]
 [ 0.          0.          0.         -0.13684211]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.26315789 -0.17894737  0.47368421 -0.15789474] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 7350 TotalParamVariation 0.00678

Epoch 8150 TotalParamVariation 0.00378947
[[ 0.         -0.45263158  0.24210526 -0.26315789]
 [ 0.          0.         -0.2        -0.28421053]
 [ 0.          0.          0.         -0.01052632]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.28421053 -0.2         0.62105263 -0.01052632] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 8200 TotalParamVariation 0.00631579
[[ 0.         -0.49473684  0.24210526 -0.26315789]
 [ 0.          0.         -0.41052632 -0.2       ]
 [ 0.          0.          0.          0.03157895]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.22105263 -0.43157895  0.6         0.05263158] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 8250 TotalParamVariation 0.00873

Epoch 9050 TotalParamVariation 0.01473684
[[ 0.         -0.55789474  0.51578947 -0.15789474]
 [ 0.          0.         -0.53684211 -0.28421053]
 [ 0.          0.          0.          0.07368421]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.38947368 -0.49473684  0.53684211  0.24210526] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 9100 TotalParamVariation 0.01057895
[[ 0.         -0.64210526  0.32631579 -0.41052632]
 [ 0.          0.         -0.22105263  0.05263158]
 [ 0.          0.          0.         -0.03157895]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.17894737 -0.15789474  0.51578947 -0.13684211] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 9150 TotalParamVariation 0.00778

Epoch 9950 TotalParamVariation 0.01310526
[[ 0.         -0.45263158  0.05263158 -0.38947368]
 [ 0.          0.         -0.09473684 -0.11578947]
 [ 0.          0.          0.          0.05263158]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.43157895 -0.17894737  0.36842105 -0.2       ] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Epoch 9999 TotalParamVariation 0.005
[[ 0.         -0.49473684  0.2        -0.26315789]
 [ 0.          0.         -0.2        -0.24210526]
 [ 0.          0.          0.          0.11578947]
 [ 0.          0.          0.          0.        ]] Corr Model
[[ 0.   -0.5   0.25 -0.25]
 [ 0.    0.   -0.25 -0.25]
 [ 0.    0.    0.    0.  ]
 [ 0.    0.    0.    0.  ]] Corr Data
[ 0.32631579 -0.24210526  0.62105263 -0.05263158] Average Model
[ 0.25 -0.25  0.5   0.  ] Average Data

Final stats. TotalParamVariation 0.005
Tota

In [25]:
a = [0.9107870628157124, 0.9710475931636243, -0.9682787056576644, -1.0325613050491655, 1.0002218579424096, 1.0000854191038566, -1.0028838357897643, -1.0029590186436916, -1.0003863975670955, -1.0000656942225046, 1.127784980225538, 1.1276617774188478, -0.9709858494773228, -0.9103129982604652, 1.0323862030607138, 0.9680813132145235]

In [27]:
sum(a)

0.24962240227755228